In [2]:
import scipy.signal as signal
from sklearn.model_selection import KFold, train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, LambdaCallback
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import random

import os
import numpy as np
from scipy import signal

from keras import layers, models, regularizers

from tensorflow.keras import layers, models, regularizers

def create_model(config):
    kernel_size = config.kernel_size
    regularizer = regularizers.l2(config.reg)
    dropout_rate = config.dropout
    size_0 = config.start_filters

    # Input layer
    in_data = layers.Input(shape=(config.input_size, 1))

    # Encoder part
    conv0 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(in_data)
    conv0 = layers.BatchNormalization()(conv0)
    conv0 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv0)
    conv0 = layers.BatchNormalization()(conv0)
    pool0 = layers.MaxPooling1D(pool_size=2)(conv0)

    size_1 = size_0 * 2
    conv1 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(pool0)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.Dropout(dropout_rate)(conv1)
    pool1 = layers.MaxPooling1D(pool_size=2)(conv1)

    size_2 = size_1 * 2
    conv2 = layers.Conv1D(size_2, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.Conv1D(size_2, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = layers.BatchNormalization()(conv2)

    # Decoder part
    up1 = layers.UpSampling1D(size=2)(conv2)
    up_conv1 = layers.Conv1D(size_2, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up1)
    up_conv1 = layers.BatchNormalization()(up_conv1)
    merge1 = layers.concatenate([conv1, up_conv1], axis=2)
    conv3 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(merge1)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Conv1D(size_1, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.Dropout(dropout_rate)(conv3)

    up2 = layers.UpSampling1D(size=2)(conv3)
    up_conv2 = layers.Conv1D(size_1, 2, activation='relu', padding='same', kernel_initializer='he_normal')(up2)
    up_conv2 = layers.BatchNormalization()(up_conv2)
    merge2 = layers.concatenate([conv0, up_conv2], axis=2)
    conv4 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(merge2)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.Conv1D(size_0, kernel_size, kernel_regularizer=regularizer, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = layers.BatchNormalization()(conv4)

    # Output layer
    out_data = layers.Conv1D(1, kernel_size, activation='sigmoid', padding='same')(conv4)

    model = models.Model(inputs=[in_data], outputs=[out_data])
    model.summary()

    return model


def sliding_window(data, window_size, downsampled_window_size, overlap, train_patients, validation_patients, test_patients):
    windows_ecg_train = []
    windows_resp_train = []

    for train_patient in train_patients:
    
        N = len(data[train_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[train_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[train_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_train.append(np.float32(resampled_ecg))
                    windows_resp_train.append(np.float32(resampled_resp))
            
            
    windows_ecg_validation = []
    windows_resp_validation = []


    for validation_patient in validation_patients:
        N = len(data[validation_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[validation_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[validation_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_validation.append(np.float32(resampled_ecg))
                    windows_resp_validation.append(np.float32(resampled_resp))
          
    windows_ecg_test = []
    windows_resp_test = []
    
    for test_patient in test_patients:
        N = len(data[test_patient][0])
        max_step = int(N//(window_size*overlap))
        for step in range(max_step):
            ecg = data[test_patient][0][step * int(window_size*overlap):step * int(window_size*overlap) + window_size] 
            resp = data[test_patient][1][step * int(window_size*overlap):step * int(window_size*overlap) + window_size]
            
            if (ecg.min() < ecg.max()):
                normalized_ecg = (ecg-ecg.min())/(ecg.max()-ecg.min())-0.5
                #zero_centered_ecg = ecg - np.mean(ecg)
                #normalized_ecg = zero_centered_ecg / np.std(zero_centered_ecg)
                resampled_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                if resp.min() < resp.max():
                    normalized_resp = (resp-resp.min())/(resp.max()-resp.min())
                    #zero_centered_resp = resp - np.mean(resp)
                    #normalized_resp = zero_centered_resp / np.std(zero_centered_resp)
                    resampled_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_ecg_test.append(np.float32(resampled_ecg))
                    windows_resp_test.append(np.float32(resampled_resp))

    windows_ecg_train = np.stack(windows_ecg_train, axis=0)
    windows_resp_train = np.stack(windows_resp_train, axis=0)
    windows_ecg_validation = np.stack(windows_ecg_validation, axis=0)
    windows_resp_validation = np.stack(windows_resp_validation, axis=0)
    windows_ecg_test = np.stack(windows_ecg_test, axis=0)
    windows_resp_test = np.stack(windows_resp_test, axis=0)

    windows_ecg_train = windows_ecg_train[:,:,np.newaxis]
    windows_resp_train = windows_resp_train[:,:,np.newaxis]
    windows_ecg_validation = windows_ecg_validation[:,:,np.newaxis]
    windows_resp_validation = windows_resp_validation[:,:,np.newaxis]
    windows_ecg_test = windows_ecg_test[:,:,np.newaxis]
    windows_resp_test = windows_resp_test[:,:,np.newaxis]

    return windows_ecg_train, windows_resp_train, windows_ecg_validation, windows_resp_validation, windows_ecg_test, windows_resp_test


def process_data_segment(data, downsampled_window_size, patient_indices):
    window_size = int(downsampled_window_size * 0.9765625)
    overlap = 1 / 2

    windows_ecg = []
    windows_resp = []

    for record_index in patient_indices:
        N = len(data[record_index][0, :])
        max_step = int(N // (window_size * overlap))
        for step in range(1, max_step - 1):
            start_idx = int(step * window_size * overlap)
            end_idx = start_idx + window_size
            recrd_ecg = data[record_index][0, start_idx:end_idx]
            recrd_resp = data[record_index][1, start_idx:end_idx]

            if recrd_ecg.min() < recrd_ecg.max():  # Skip flatlined signal segments
                normalized_ecg = (recrd_ecg - recrd_ecg.min()) / (recrd_ecg.max() - recrd_ecg.min())
                normalized_ecg = signal.resample(normalized_ecg, downsampled_window_size)
                windows_ecg.append(np.float16(normalized_ecg))

                if recrd_resp.min() < recrd_resp.max():
                    normalized_resp = (recrd_resp - recrd_resp.min()) / (recrd_resp.max() - recrd_resp.min())
                    normalized_resp = signal.resample(normalized_resp, downsampled_window_size)
                    windows_resp.append(np.float16(normalized_resp))

    windows_ecg = np.array(windows_ecg)[:, :, np.newaxis]
    windows_resp = np.array(windows_resp)[:, :, np.newaxis]

    print(windows_ecg.shape)
    print(windows_resp.shape)

    return windows_ecg, windows_resp


def load_data():
     # bidmc
    path = "/Users/lanacaldarevic/workspace/phd/ecg_derived_resp_dl/data/bidmc-ppg-and-respiration-dataset-1.0.0"
    EXT = "*Signals.csv"
    all_csv_files = [file for path, subdir, files in os.walk(path) for file in glob(os.path.join(path, EXT))]
    patients = []
    data = {}
    no_errors = 0
    for file in all_csv_files:
        try:
            df = pd.read_csv(file)
            X1, X2, X3, X4 = df[' PLETH'].values, df[' V'].values, df[' AVR'].values, df[' II'].values
            # X = np.concatenate([X1.reshape(len(X1),1),X2.reshape(len(X1),1),X3.reshape(len(X1),1),X4.reshape(len(X1),1)], axis=1)
            
            Y = df[' RESP'].values
            
            patient = int(file.split('/')[-1].split('_')[1])
            patients.append(patient)
            #data[patient] = [X4, Y]
            data[patient] = np.array([X4, Y])
        except:
            no_errors += 1

    return data, patients

In [3]:
class VisualiseCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        fig, ax = plt.subplots(2,2)
        fig.suptitle('Ground truth vs. Prediction')
        for row in range(2):
            train_idx = random.randint(0, windows_ecg_train[:,:,:].shape[0]-1)
            val_idx = random.randint(0, windows_ecg_validation[:,:,:].shape[0]-1)
            to_predict_train = np.array([windows_ecg_train[train_idx,:,:]])
            ground_truth_train = windows_resp_train[train_idx,:,:]
            prediction_train = self.model.predict(to_predict_train)[0] 
            prediction_train_score=self.model.evaluate(to_predict_train,np.array([ground_truth_train]))
            to_predict_validation = np.array([windows_ecg_validation[val_idx,:,:]])
            ground_truth_validation = windows_resp_validation[val_idx,:,:]
            prediction_validation = self.model.predict(to_predict_validation)[0]
            prediction_validation_score = self.model.evaluate(to_predict_validation, np.array([ground_truth_validation]))
            ax[row,0].plot(ground_truth_train, label='Ground Truth')
            ax[row,0].plot(prediction_train, label='Prediction')
            ax[row,0].set_title("Train L "+str(prediction_train_score[0])[:6]+" CC "+str(prediction_train_score[2])[:6])
            ax[row,0].legend()
            
            ax[row,1].plot(ground_truth_validation, label='Ground Truth')
            ax[row,1].plot(prediction_validation, label='Prediction')
            ax[row,1].set_title("Valid L "+str(prediction_validation_score[0])[:6]+" CC "+str(prediction_validation_score[2])[:6])
            ax[row,1].legend()
            
            plt.tight_layout()
            
            
        wandb.log({
            "predictions_visualization": wandb.Image(fig)

        }, commit=False)
        plt.close(fig)

def correlation(x, y): #todo: check this and see in papers what cross correlation is
    # Normalize y to the [0, 1] range
    min_y = tf.math.reduce_min(y)
    max_y = tf.math.reduce_max(y)
    r_up = tf.math.subtract(y, min_y)
    r_down = max_y - min_y
    new_y = r_up / r_down
    
    # Compute means
    mx = tf.math.reduce_mean(x)
    my = tf.math.reduce_mean(new_y)
    
    # Compute centered values
    xm, ym = x - mx, new_y - my
    
    # Compute correlation coefficient
    r_num = tf.reduce_sum(tf.multiply(xm, ym))
    r_den = tf.sqrt(tf.multiply(tf.reduce_sum(tf.square(xm)), tf.reduce_sum(tf.square(ym))))
    r = r_num / r_den
    
    # Ensure the result is between -1 and 1
    r = tf.maximum(tf.minimum(r, 1.0), -1.0)
    
    return 1 - r

def cross_correlation(y_true, y_pred):
    """ Compute cross-correlation between true and predicted signals. """
    y_true_mean = tf.reduce_mean(y_true, axis=1, keepdims=True)
    y_pred_mean = tf.reduce_mean(y_pred, axis=1, keepdims=True)
    
    y_true_std = tf.math.reduce_std(y_true, axis=1, keepdims=True)
    y_pred_std = tf.math.reduce_std(y_pred, axis=1, keepdims=True)

    norm_y_true = (y_true - y_true_mean) / y_true_std
    norm_y_pred = (y_pred - y_pred_mean) / y_pred_std

    correlation = tf.reduce_mean(norm_y_true * norm_y_pred, axis=1)
    return tf.reduce_mean(correlation)

In [17]:
import datetime
def train(config):
    lr = config.learning_rate
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    
    model = create_model(config)
    
    # define callbacks
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.01, patience=10, cooldown=5, mode='min', min_lr=0.0000001)
    filepath = os.path.join('CV_results', f'model_crossval{config.split_ind}-size{config.start_filters}-input{config.input_size}-lr{config.learning_rate}-kernel{config.kernel_size}-reg{config.reg}-dropout{config.dropout}.h5')
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    
    # early stopping
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, start_from_epoch=20, restore_best_weights=True)
    

    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='epoch')
    callbacks = [checkpoint, early_stopping]

    """ 
    # Hubner
    loss = tf.keras.losses.Huber()
    model.compile(loss=loss, metrics=[correlation, 'mse'], optimizer=optimizer)
    """

    #optimizer = tf.keras.optimizers.legacy.RMSprop(lr=lr, momentum=0.7)
    optimizer = tf.keras.optimizers.legacy.SGD(lr=lr, momentum=0.7)

    model.compile(loss='mse', metrics=['mse', correlation, cross_correlation], optimizer=optimizer)
    
    print("Model training starting")
    model.fit(windows_ecg_train, windows_resp_train,
              epochs=config.epochs,
              batch_size=config.batch_size,
              shuffle=True,
              callbacks=callbacks,
              validation_data=(windows_ecg_validation, windows_resp_validation))
    
    model.save(os.path.join('CV_results', f'combined_model{config.split_ind}-size{config.start_filters}-input{config.input_size}-lr{config.learning_rate}-kernel{config.kernel_size}-reg{config.reg}-dropout{config.dropout}.h5'))

    return model

In [18]:
sampling_rate = 125
input_size_seconds = 16 # //2, *2
downsampled_window_size = 1024 #? power of 2
window_size = input_size_seconds * sampling_rate
overlap = 0.5 #25%

data, patients = load_data()

In [19]:
np.random.seed(42)
unique_patients = list(set(patients))
np.random.shuffle(unique_patients)

train_val_patients, test_patients = train_test_split(unique_patients, test_size=0.20, random_state=42)

In [20]:
start_filters = 8
input_size = downsampled_window_size

In [21]:
class ModelConfig:
    def __init__(self, learning_rate, epochs, batch_size, start_filters, input_size, kernel_size, reg, dropout, split_ind):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.start_filters = start_filters
        self.input_size = input_size
        self.kernel_size = kernel_size
        self.reg = reg
        self.dropout = dropout
        self.split_ind = split_ind


In [ ]:
results_mse = []
results_corr = []
results_cc = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for split_index, (train_index, val_index) in enumerate(kf.split(train_val_patients)):
    train_patients = [train_val_patients[i] for i in train_index]
    val_patients = [train_val_patients[i] for i in val_index]

    windows_ecg_train, windows_resp_train = process_data_segment(data, downsampled_window_size, train_patients)
    windows_ecg_validation, windows_resp_validation = process_data_segment(data, downsampled_window_size, val_patients)

    config = {
        "learning_rate": 0.004878689057515219,
        "epochs": 500,
        "batch_size": 256,
        "start_filters": start_filters,
        "input_size": input_size,
        "kernel_size": 27,
        "reg": 0.006032564292289038,
        "dropout": 0.8804727366138194,
        "split_ind": split_index
    }
    config = ModelConfig(**config)
    model = train(config)
    _, mse, corr, cc = model.evaluate(windows_ecg_validation, windows_resp_validation)
    results_mse.append(mse)
    results_corr.append(corr)
    results_cc.append(cc)

# Output the results from all folds


(3304, 1024, 1)
(3304, 1024, 1)
(944, 1024, 1)
(944, 1024, 1)
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 1024, 1)]            0         []                            
                                                                                                  
 conv1d_65 (Conv1D)          (None, 1024, 8)              224       ['input_6[0][0]']             
                                                                                                  
 batch_normalization_60 (Ba  (None, 1024, 8)              32        ['conv1d_65[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 conv1d_66 (Conv1D)          (

/Users/lanacaldarevic/miniforge3/envs/tf_m1/lib/python3.9/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/500


2024-04-17 19:55:50.676891: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - ETA: 0s - loss: 2.1119 - mse: 0.1887 - correlation: 1.0000 - cross_correlation: 0.0057

...

13/13 [==============================] - ETA: 0s - loss: 2.0991 - mse: 0.1853 - correlation: 0.9995 - cross_correlation: 0.0066
Epoch 2: val_loss improved from 2.02376 to 2.01309, saving model to CV_results/model_crossval0-size8-input1024-lr0.004878689057515219-kernel27-reg0.006032564292289038-dropout0.8804727366138194.h5
13/13 [==============================] - 11s 798ms/step - loss: 2.0991 - mse: 0.1853 - correlation: 0.9995 - cross_correlation: 0.0066 - val_loss: 2.0131 - val_mse: 0.1046 - val_correlation: 0.9948 - val_cross_correlation: 0.0226
Epoch 3/500
13/13 [==============================] - ETA: 0s - loss: 2.0863 - mse: 0.1822 - correlation: 1.0006 - cross_correlation: 0.0050
Epoch 3: val_loss improved from 2.01309 to 2.00341, saving model to CV_results/model_crossval0-size8-input1024-lr0.004878689057515219-kernel27-reg0.006032564292289038-dropout0.8804727366138194.h5
13/13 [==============================] - 10s 792ms/step - loss: 2.0863 - mse: 0.1822 - correlation: 1.0006 - c

2024-04-17 21:07:24.747494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - ETA: 0s - loss: 2.1064 - mse: 0.1850 - correlation: 0.9966 - cross_correlation: 0.0014

2024-04-17 21:08:09.030824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 2.04821, saving model to CV_results/model_crossval1-size8-input1024-lr0.004878689057515219-kernel27-reg0.006032564292289038-dropout0.8804727366138194.h5
14/14 [==============================] - 65s 3s/step - loss: 2.1064 - mse: 0.1850 - correlation: 0.9966 - cross_correlation: 0.0014 - val_loss: 2.0482 - val_mse: 0.1323 - val_correlation: 1.0139 - val_cross_correlation: 0.0186
Epoch 2/500
14/14 [==============================] - ETA: 0s - loss: 2.0931 - mse: 0.1819 - correlation: 0.9952 - cross_correlation: 0.0027
Epoch 2: val_loss improved from 2.04821 to 2.03781, saving model to CV_results/model_crossval1-size8-input1024-lr0.004878689057515219-kernel27-reg0.006032564292289038-dropout0.8804727366138194.h5
14/14 [==============================] - 9s 619ms/step - loss: 2.0931 - mse: 0.1819 - correlation: 0.9952 - cross_correlation: 0.0027 - val_loss: 2.0378 - val_mse: 0.1324 - val_correlation: 1.0355 - val_cross_correlation: 0.0194
Epoch 3/500
14/

In [16]:
print("Average MSE performance across folds:", np.mean(results_mse))
print("Average CORR performance across folds:", np.mean(results_corr))
print("Average CC performance across folds:", np.mean(results_cc))

Average MSE performance across folds: 0.12231316715478897
Average CORR performance across folds: 0.9378519415855407
Average CC performance across folds: 0.0539246825966984


In [ ]:
# check visually
def plot_signals_in_grid(predicted_signals, true_signals, labels=['Predicted', 'True']):
    fig, axs = plt.subplots(len(predicted_signals)//2, 2, figsize=(20, 50))
    axs = axs.flatten() 

    for i in range(len(predicted_signals)):  
        ax = axs[i]
        pred_signal = predicted_signals[i]
        true_signal = true_signals[i]

        # Plotting both signals in the same subplot
        ax.plot(pred_signal, label=labels[0])
        ax.plot(true_signal, label=labels[1], alpha=0.75)
        ax.legend()
        ax.set_title(f'Example {i+1}')
    
    plt.tight_layout()
    plt.show()

In [24]:
results_cc

[-0.005258073098957539,
 0.03349137678742409,
 0.008275619708001614,
 -0.009416820481419563,
 0.108854278922081]